# Cleaning of Rat Sightings

In [1]:
import os
import requests
import json
import pandas as pd
from pandas import json_normalize
import numpy as np
import datetime

In [2]:
# importing rats data
rats_df = pd.read_csv('rat_sightings.csv')
rats_df.head()

C:\Users\annie\AppData\Local\Temp\ipykernel_16048\1420771104.py:2: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  rats_df = pd.read_csv('rat_sightings.csv')


,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,...,Vehicle Type,Taxi Company Borough,Taxi Pick Up Location,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Latitude,Longitude,Location
0,40177738,09/03/2018 12:21:09 PM,09/14/2018 05:46:05 PM,DOHMH,Department of Health and Mental Hygiene,Rodent,Rat Sighting,3+ Family Apt. Building,11229.0,1213 AVENUE U,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.598479,-73.959684,"(40.598478991333735, -73.9596835550102)"
1,52046514,10/01/2021 06:36:43 PM,NaN,DOHMH,Department of Health and Mental Hygiene,Rodent,Rat Sighting,Other (Explain Below),11222.0,NASSAU AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.725616,-73.944497,"(40.72561631687134, -73.94449701211244)"
2,39718837,07/12/2018 10:17:56 PM,07/24/2018 02:38:39 PM,DOHMH,Department of Health and Mental Hygiene,Rodent,Rat Sighting,Vacant Lot,10009.0,600 EAST 13 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.728716,-73.978485,"(40.72871553527489, -73.97848545954511)"
3,39719951,07/12/2018 09:37:14 AM,07/16/2018 04:27:49 PM,DOHMH,Department of Health and Mental Hygiene,Rodent,Rat Sighting,1-2 Family Dwelling,11416.0,101-45 103 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.685639,-73.839738,"(40.68563941599227, -73.83973845918783)"
4,39742405,07/15/2018 03:00:10 PM,07/18/2018 03:46:44 PM,DOHMH,Department of Health and Mental Hygiene,Rodent,Rat Sighting,3+ Family Mixed Use Building,11216.0,1180 BEDFORD AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.683790,-73.954007,"(40.68379029014929, -73.95400727877036)"


In [5]:
rats_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206999 entries, 0 to 206998
Data columns (total 38 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   Unique Key                      206999 non-null  int64  
 1   Created Date                    206999 non-null  object 
 2   Closed Date                     190420 non-null  object 
 3   Agency                          206999 non-null  object 
 4   Agency Name                     206999 non-null  object 
 5   Complaint Type                  206999 non-null  object 
 6   Descriptor                      206999 non-null  object 
 7   Location Type                   206987 non-null  object 
 8   Incident Zip                    206571 non-null  float64
 9   Incident Address                196066 non-null  object 
 10  Street Name                     196065 non-null  object 
 11  Cross Street 1                  180660 non-null  object 
 12  Cross Street 2  

No missing dates or unique keys, appears that there may be some missing location data, but as long as we have the ZIP, we can assign a borough & neighbourhood

In [ ]:
# checking for duplicates on Unique Key

In [11]:
rats_df[rats_df.duplicated(subset='Unique Key')]

,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,...,Vehicle Type,Taxi Company Borough,Taxi Pick Up Location,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Latitude,Longitude,Location


In [ ]:
# no duplicates found!

In [13]:
# keeping only location & date data, as well as complaint status
rats_df = rats_df[['Created Date', 'Closed Date','Status', 'Borough','Latitude','Longitude','Location', 'Incident Zip']].copy()

In [14]:
rats_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206999 entries, 0 to 206998
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Created Date  206999 non-null  object 
 1   Closed Date   190420 non-null  object 
 2   Status        206999 non-null  object 
 3   Borough       206995 non-null  object 
 4   Latitude      204844 non-null  float64
 5   Longitude     204844 non-null  float64
 6   Location      204844 non-null  object 
 7   Incident Zip  206571 non-null  float64
dtypes: float64(3), object(5)
memory usage: 12.6+ MB


In [179]:
# converting date to datetime object
rats_df['Created Date'] = pd.to_datetime(rats_df['Created Date'])
rats_df['Closed Date'] = pd.to_datetime(rats_df['Closed Date'])

In [18]:
# splitting dates into year/month/day
rats_df['Created Year'] = rats_df['Created Date'].dt.strftime('%Y')
rats_df['Created Month'] = rats_df['Created Date'].dt.strftime('%m')
rats_df['Created Day'] = rats_df['Created Date'].dt.strftime('%d')
rats_df['Closed Year'] = rats_df['Closed Date'].dt.strftime('%Y')
rats_df['Closed Month'] = rats_df['Closed Date'].dt.strftime('%m')
rats_df['Closed Day'] = rats_df['Closed Date'].dt.strftime('%d')

In [19]:
rats_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206999 entries, 0 to 206998
Data columns (total 14 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   Created Date   206999 non-null  datetime64[ns]
 1   Closed Date    190420 non-null  datetime64[ns]
 2   Status         206999 non-null  object        
 3   Borough        206995 non-null  object        
 4   Latitude       204844 non-null  float64       
 5   Longitude      204844 non-null  float64       
 6   Location       204844 non-null  object        
 7   Incident Zip   206571 non-null  float64       
 8   Created Year   206999 non-null  object        
 9   Created Month  206999 non-null  object        
 10  Created Day    206999 non-null  object        
 11  Closed Year    190420 non-null  object        
 12  Closed Month   190420 non-null  object        
 13  Closed Day     190420 non-null  object        
dtypes: datetime64[ns](2), float64(3), object(9)
memory u

In [20]:
# creating a new column that is the time between a complaint being made, and a complaint being closed
rats_df['Time Complaint Open'] = rats_df['Closed Date'] - rats_df['Created Date']

In [21]:
rats_df

,Created Date,Closed Date,Status,Borough,Latitude,Longitude,Location,Incident Zip,Created Year,Created Month,Created Day,Closed Year,Closed Month,Closed Day,Time Complaint Open
0,2018-09-03 12:21:09,2018-09-14 17:46:05,Closed,BROOKLYN,40.598479,-73.959684,"(40.598478991333735, -73.9596835550102)",11229.0,2018,09,03,2018,09,14,11 days 05:24:56
1,2021-10-01 18:36:43,NaT,In Progress,BROOKLYN,40.725616,-73.944497,"(40.72561631687134, -73.94449701211244)",11222.0,2021,10,01,NaN,NaN,NaN,NaT
2,2018-07-12 22:17:56,2018-07-24 14:38:39,Closed,MANHATTAN,40.728716,-73.978485,"(40.72871553527489, -73.97848545954511)",10009.0,2018,07,12,2018,07,24,11 days 16:20:43
3,2018-07-12 09:37:14,2018-07-16 16:27:49,Closed,QUEENS,40.685639,-73.839738,"(40.68563941599227, -73.83973845918783)",11416.0,2018,07,12,2018,07,16,4 days 06:50:35
4,2018-07-15 15:00:10,2018-07-18 15:46:44,Closed,BROOKLYN,40.683790,-73.954007,"(40.68379029014929, -73.95400727877036)",11216.0,2018,07,15,2018,07,18,3 days 00:46:34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206994,2022-11-25 13:27:32,NaT,In Progress,BRONX,40.828365,-73.908541,"(40.82836459096858, -73.90854097623259)",10456.0,2022,11,25,NaN,NaN,NaN,NaT
206995,2022-11-25 17:22:31,NaT,In Progress,BROOKLYN,40.645350,-73.970318,"(40.64534954262103, -73.97031798578249)",11218.0,2022,11,25,NaN,NaN,NaN,NaT
206996,2022-11-25 15:00:51,NaT,In Progress,MANHATTAN,40.797867,-73.967581,"(40.79786676610781, -73.9675808951411)",10025.0,2022,11,25,NaN,NaN,NaN,NaT
206997,2022-11-25 09:30:56,NaT,In Progress,MANHATTAN,40.807309,-73.940336,"(40.8073087216762, -73.94033619161439)",10035.0,2022,11,25,NaN,NaN,NaN,NaT


In [22]:
# assigning a neighbourhood to every incident

In [174]:
# converting numbers to strings so we can search using them as our substring value
rats_df['Incident Zip'] = rats_df['Incident Zip'].astype(str)
# removing trailing '.0'
rats_df['Incident Zip'] = rats_df['Incident Zip'].str.replace('\.0','')

C:\Users\annie\AppData\Local\Temp\ipykernel_16048\3881869699.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  rats_df['Incident Zip'] = rats_df['Incident Zip'].str.replace('\.0','')


In [26]:
rats_df.head()

,Created Date,Closed Date,Status,Borough,Latitude,Longitude,Location,Incident Zip,Created Year,Created Month,Created Day,Closed Year,Closed Month,Closed Day,Time Complaint Open
0,2018-09-03 12:21:09,2018-09-14 17:46:05,Closed,BROOKLYN,40.598479,-73.959684,"(40.598478991333735, -73.9596835550102)",11229,2018,09,03,2018,09,14,11 days 05:24:56
1,2021-10-01 18:36:43,NaT,In Progress,BROOKLYN,40.725616,-73.944497,"(40.72561631687134, -73.94449701211244)",11222,2021,10,01,NaN,NaN,NaN,NaT
2,2018-07-12 22:17:56,2018-07-24 14:38:39,Closed,MANHATTAN,40.728716,-73.978485,"(40.72871553527489, -73.97848545954511)",10009,2018,07,12,2018,07,24,11 days 16:20:43
3,2018-07-12 09:37:14,2018-07-16 16:27:49,Closed,QUEENS,40.685639,-73.839738,"(40.68563941599227, -73.83973845918783)",11416,2018,07,12,2018,07,16,4 days 06:50:35
4,2018-07-15 15:00:10,2018-07-18 15:46:44,Closed,BROOKLYN,40.683790,-73.954007,"(40.68379029014929, -73.95400727877036)",11216,2018,07,15,2018,07,18,3 days 00:46:34


In [30]:
# importing csvs
nyc_bo = pd.read_csv('nyc_boroughs.csv') # boroughs
nyc_nb = pd.read_csv('nyc_neighbs.csv',encoding='cp1252') # neighbourhoods
# already know we will have to reformat the data a little, so just getting that done at the same time
# converting numbers to strings so we can search using them as our substring value
nyc_bo = nyc_bo.astype(str)
nyc_nb = nyc_nb.astype(str)
# removing trailing '.0'
nyc_bo = nyc_bo.stack().str.replace(r'\.0','').unstack()
nyc_nb = nyc_nb.stack().str.replace(r'\.0','').unstack()

C:\Users\annie\AppData\Local\Temp\ipykernel_16048\1744579740.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  nyc_bo = nyc_bo.stack().str.replace(r'\.0','').unstack()
C:\Users\annie\AppData\Local\Temp\ipykernel_16048\1744579740.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  nyc_nb = nyc_nb.stack().str.replace(r'\.0','').unstack()


This data already has boroughs assigned, but we will be re-running it through our own borough data anyway for two reasons:
1. Consistency across data
    - Zipcodes in all of our data should be pointing to the same neighbourhoods & boroughs
2. Formatting
    - The boroughs in this data are all written in allcaps, and this conveniently will re-format casing without needing to use `str.title()`

In [32]:
nyc_bo.head()

,Manhattan,Staten Island,Bronx,Queens,Brooklyn
0,10001,10301,10451,11004,11201
1,10002,10302,10452,11005,11202
2,10003,10303,10453,11006,11203
3,10004,10304,10454,11007,11204
4,10005,10305,10455,11008,11205


In [33]:
nyc_nb.head()

,Port Richmond,South Beach and Tottenville,Stapleton and St. George,Mid-Island (Willowbrook),Inwood and Washington Heights,Central Harlem,East Harlem,Upper West Side,Upper East Side,Chelsea and Clinton,...,Northwest Brooklyn,Central Brooklyn (Bedford Stuyvesant – Crown Heights),East New York and New Lots,Sunset Park,Southwest Brooklyn,Borough Park,Canarsie and Flatlands,Southern Brooklyn,Flatbush,Bushwick and Williamsburg
0,10302,10306,10301,10314,10031,10026,10029,10023,10021,10001,...,11201,11212,11207,11220,11209,11204,11234,11223,11203,11206
1,10303,10307,10304,nan,10032,10027,10035,10024,10028,10011,...,11205,11213,11208,11232,11214,11218,11236,11224,11210,11221
2,10310,10308,10305,nan,10033,10030,nan,10025,10044,10018,...,11215,11216,nan,nan,11228,11219,11239,11229,11225,11237
3,nan,nan,nan,nan,10034,10037,nan,nan,10065,10019,...,11217,11233,nan,nan,nan,11230,nan,11235,11226,nan
4,nan,nan,nan,nan,10040,10039,nan,nan,10075,10020,...,11231,11238,nan,nan,nan,nan,nan,nan,nan,nan


In [37]:
# creating lists of borough & neighbourhood names
bo_list = nyc_bo.columns.tolist()
nb_list = nyc_nb.columns.tolist()

In [45]:
print('Boroughs:')
print(bo_list)
print('Neighbourhoods:')
print(nb_list)

Boroughs:
['Manhattan', 'Staten Island', 'Bronx', 'Queens', 'Brooklyn']
Neighbourhoods:
['Port Richmond', 'South Beach and Tottenville', 'Stapleton and St. George', 'Mid-Island (Willowbrook)', 'Inwood and Washington Heights', 'Central Harlem', 'East Harlem', 'Upper West Side', 'Upper East Side', 'Chelsea and Clinton', 'Gramercy Park and Murray Hill', 'Greenwich Village and Soho', 'Union Square and Lower East Side', 'Lower Manhattan', 'Kingsbridge and Riverdale', 'Northeast Bronx', 'Crotona and Tremont', 'Bronx Park and Fordham', 'Pelham and Throgs Neck', 'HighBridge and Morrisania', 'Hunts Point and Mott Haven', 'Long Island City and Astoria', 'West Queens', 'North Queens', 'Northeast Queens', 'West Central Queens', 'Central Queens', 'Southwest Queens', 'Jamaica', 'Southeast Queens', 'Rockaways', 'Greenpoint', 'Northwest Brooklyn ', 'Central Brooklyn (Bedford Stuyvesant – Crown Heights)', 'East New York and New Lots', 'Sunset Park', 'Southwest Brooklyn', 'Borough Park', 'Canarsie and F

In [67]:
def assign_borough(borough):
    """ Finds borough name for address dataframe.
    
    Loops through the nyc_bo (nyc boroughs) dataframe to see if one of its zipcodes 
    is within the address string of our addresses dataframe. Then fills in borough name
    
    Params:
        borough: desired borough (column from nyc_bo dataframe)
        
    Returns:
        fills 'Borough' value with column name from nyc_bo
    
    """
    num_rows = nyc_bo.shape[0]
    row_count = 0

    while row_count < num_rows:
        rats_df.loc[rats_df['Incident Zip'] ==nyc_bo[borough][row_count], 'Borough'] = borough
        #rats_df.loc[rats_df['Incident Zip'] ==nyc_bo[borough][row_count], 'Incident Zip'] = nyc_bo[borough][row_count]
        row_count +=1

In [69]:
# loop to run borough assignment
"""
iterates through list of borough names & inserts them runs assign_borough function every value
"""
for i in range(len(bo_list)):
    assign_borough(bo_list[i])

In [71]:
# looks like some of the locations got missed, will have to go in and investigate
rats_df['Borough'].unique()

array(['Brooklyn', 'Manhattan', 'Queens', 'Bronx', 'Staten Island',
       'MANHATTAN', 'QUEENS', 'BROOKLYN'], dtype=object)

In [73]:
def assign_neighbourhood(neighbourhood):
    """ Finds neighbourhood name for address dataframe.
    
    Loops through the nyc_bo (nyc boroughs) dataframe to see if one of its zipcodes 
    is within the address string of our addresses dataframe. Then fills in borough name & cleanly formatted zipcode
    
    Params:
        borough: desired neighbourhood (column from nyc_bo dataframe)
        
    Returns:
        fills 'Neighbourhood' value with column name from nyc_nb
    
    """
    num_rows = nyc_nb.shape[0]
    row_count = 0

    while row_count < num_rows:
        rats_df.loc[rats_df['Incident Zip'] ==nyc_nb[neighbourhood][row_count], 'Neighbourhood'] = neighbourhood
        rats_df.loc[rats_df['Incident Zip'] ==nyc_nb[neighbourhood][row_count], 'Incident Zip'] = nyc_nb[neighbourhood][row_count]
        row_count +=1

In [74]:
# loop to run neighbourhood assignment
"""
iterates through list of neighbourhood names & inserts them runs assign_neighbourhood function every value
"""
for i in range(len(nb_list)):
    assign_neighbourhood(nb_list[i])

In [88]:
rats_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206999 entries, 0 to 206998
Data columns (total 16 columns):
 #   Column               Non-Null Count   Dtype          
---  ------               --------------   -----          
 0   Created Date         206999 non-null  datetime64[ns] 
 1   Closed Date          190420 non-null  datetime64[ns] 
 2   Status               206999 non-null  object         
 3   Borough              206999 non-null  object         
 4   Latitude             204844 non-null  float64        
 5   Longitude            204844 non-null  float64        
 6   Location             204844 non-null  object         
 7   Incident Zip         206999 non-null  object         
 8   Created Year         206999 non-null  object         
 9   Created Month        206999 non-null  object         
 10  Created Day          206999 non-null  object         
 11  Closed Year          190420 non-null  object         
 12  Closed Month         190420 non-null  object         
 13 

In [100]:
# such a small percentage missing that I'm just going to drop these
rats_df.dropna(subset='Neighbourhood', inplace=True)

In [101]:
rats_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 204474 entries, 0 to 206998
Data columns (total 16 columns):
 #   Column               Non-Null Count   Dtype          
---  ------               --------------   -----          
 0   Created Date         204474 non-null  datetime64[ns] 
 1   Closed Date          188076 non-null  datetime64[ns] 
 2   Status               204474 non-null  object         
 3   Borough              204474 non-null  object         
 4   Latitude             202346 non-null  float64        
 5   Longitude            202346 non-null  float64        
 6   Location             202346 non-null  object         
 7   Incident Zip         204474 non-null  object         
 8   Created Year         204474 non-null  object         
 9   Created Month        204474 non-null  object         
 10  Created Day          204474 non-null  object         
 11  Closed Year          188076 non-null  object         
 12  Closed Month         188076 non-null  object         
 13 

In [102]:
# this conveniently also got rid of any Boroughs that were all caps, with no need to convert anything
rats_df['Borough'].unique()

array(['Brooklyn', 'Manhattan', 'Queens', 'Bronx', 'Staten Island'],
      dtype=object)

In [133]:
# looking at complaint types
rats_df['Status'].unique()

array(['Closed', 'In Progress', 'Pending', 'Assigned', 'Unspecified',
       'Open', 'Draft'], dtype=object)

In [ ]:
# removing any that are 'Unspecified' or 'Draft', as we don't know if that means if they are actually open or closed

In [149]:
rats_df.drop(rats_df[rats_df['Status'] == 'Unspecified'].index, inplace=True)
rats_df.drop(rats_df[rats_df['Status'] == 'Draft'].index, inplace=True)

In [171]:
# Replacing Status with either 'Open' or 'Closed'
closed = rats_df['Status'] == 'Closed'
rats_df['Status'].where(closed, other='Open', inplace=True)

In [178]:
rats_df.head()

,Created Date,Closed Date,Status,Borough,Latitude,Longitude,Location,Incident Zip,Created Year,Created Month,Created Day,Closed Year,Closed Month,Closed Day,Time Complaint Open,Neighbourhood
0,2018-09-03 12:21:09,2018-09-14 17:46:05,Closed,Brooklyn,40.598479,-73.959684,"(40.598478991333735, -73.9596835550102)",11229,2018,9,3,2018.0,9.0,14.0,11 days 05:24:56,Southern Brooklyn
1,2021-10-01 18:36:43,NaN,Open,Brooklyn,40.725616,-73.944497,"(40.72561631687134, -73.94449701211244)",11222,2021,10,1,NaN,NaN,NaN,NaN,Greenpoint
2,2018-07-12 22:17:56,2018-07-24 14:38:39,Closed,Manhattan,40.728716,-73.978485,"(40.72871553527489, -73.97848545954511)",10009,2018,7,12,2018.0,7.0,24.0,11 days 16:20:43,Union Square and Lower East Side
3,2018-07-12 09:37:14,2018-07-16 16:27:49,Closed,Queens,40.685639,-73.839738,"(40.68563941599227, -73.83973845918783)",11416,2018,7,12,2018.0,7.0,16.0,4 days 06:50:35,Southwest Queens
4,2018-07-15 15:00:10,2018-07-18 15:46:44,Closed,Brooklyn,40.683790,-73.954007,"(40.68379029014929, -73.95400727877036)",11216,2018,7,15,2018.0,7.0,18.0,3 days 00:46:34,Central Brooklyn (Bedford Stuyvesant – Crown H...


In [181]:
rats_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 204471 entries, 0 to 204473
Data columns (total 16 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   Created Date         204471 non-null  datetime64[ns]
 1   Closed Date          188076 non-null  datetime64[ns]
 2   Status               204471 non-null  object        
 3   Borough              204471 non-null  object        
 4   Latitude             202343 non-null  float64       
 5   Longitude            202343 non-null  float64       
 6   Location             202343 non-null  object        
 7   Incident Zip         204471 non-null  object        
 8   Created Year         204471 non-null  int64         
 9   Created Month        204471 non-null  int64         
 10  Created Day          204471 non-null  int64         
 11  Closed Year          188076 non-null  float64       
 12  Closed Month         188076 non-null  float64       
 13  Closed Day    

In [182]:
# this is a lot of data, let's slim it down
# taking a look at number of incidents per borough
rats_df.groupby(['Borough']).count()

,Created Date,Closed Date,Status,Latitude,Longitude,Location,Incident Zip,Created Year,Created Month,Created Day,Closed Year,Closed Month,Closed Day,Time Complaint Open,Neighbourhood
Borough,,,,,,,,,,,,,,,
Bronx,38664,36210,38664,38497,38497,38497,38664,38664,38664,38664,36210,36210,36210,36210,38664
Brooklyn,73866,67293,73866,72808,72808,72808,73866,73866,73866,73866,67293,67293,67293,67293,73866
Manhattan,54417,50152,54417,53742,53742,53742,54417,54417,54417,54417,50152,50152,50152,50152,54417
Queens,30351,27791,30351,30142,30142,30142,30351,30351,30351,30351,27791,27791,27791,27791,30351
Staten Island,7173,6630,7173,7154,7154,7154,7173,7173,7173,7173,6630,6630,6630,6630,7173


In [183]:
# pretty unequal, which was expected
# don't want to take an equal amount of samples per group because we want to know who has the most amount of rats
# taking a random sample from every borough of 15% (want to keep the smallest group (Staten Island) > 1000)
rats_sample = rats_df.groupby('Borough').sample(frac=.15, random_state=1)

In [184]:
# taking a look at our sample
rats_sample.groupby('Borough').count()

,Created Date,Closed Date,Status,Latitude,Longitude,Location,Incident Zip,Created Year,Created Month,Created Day,Closed Year,Closed Month,Closed Day,Time Complaint Open,Neighbourhood
Borough,,,,,,,,,,,,,,,
Bronx,5800,5425,5800,5779,5779,5779,5800,5800,5800,5800,5425,5425,5425,5425,5800
Brooklyn,11080,10107,11080,10931,10931,10931,11080,11080,11080,11080,10107,10107,10107,10107,11080
Manhattan,8163,7473,8163,8063,8063,8063,8163,8163,8163,8163,7473,7473,7473,7473,8163
Queens,4553,4182,4553,4524,4524,4524,4553,4553,4553,4553,4182,4182,4182,4182,4553
Staten Island,1076,1002,1076,1065,1065,1065,1076,1076,1076,1076,1002,1002,1002,1002,1076


In [187]:
# exporting csvs of complete data & sampled data both
rats_df.to_csv('rat_sightings_total.csv', index=False)
rats_sample.to_csv('rat_sightings_sample.csv', index=False)